In [108]:
import sys
import os
from pathlib import Path
try:
    import pyvips  # type: ignore
    _HAS_PYVIPS = True
except Exception:
    pyvips = None
    _HAS_PYVIPS = False

PROJECT_ROOT = Path("/home/users/ntu/lizh0106/scratch/nscc_work/Baseline_models")
os.chdir(PROJECT_ROOT)
#sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from src.config import load_paths, load_params
from src.data.features import prepare_features
from src.models.train import cross_validate, predict_external
from src.evaluation.grading import gleason_to_isup
from src.evaluation.metrics import evaluate_slide_predictions

# CONFIG

In [109]:
base_path =Path("results/hard_labels")

# ===================================
# test_folder = "XGB_nol2_with2048" # 需要和base_path链接，里面有aggc_prob_p 和tcga_prob_p

# df_aggc_index = "/home/users/ntu/lizh0106/scratch/nscc_work/Processed_Features/AGGC/20x_512/index.csv"
# df_tcga_index = "/home/users/ntu/lizh0106/scratch/nscc_work/Processed_Features/TGGA_PRAD_V2/TGGA_PRAD_V2_without_anno/TCGA_20x512/index.csv"

# aggc_prob_p = "aggc_oof_tile_proba.npy"
# tcga_prob_p = "tcga_proba_mean.npy"


In [ ]:
# exps 是 base_path里所有文件夹名字
import json
exps = [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]

metrics = ["acc_valid","balanced_acc_valid","classification_report"]

for experiments in exps:
    path_aggc = os.path.join(base_path, experiments, "metrics_aggc.json")
    path_tcga= os.path.join(base_path, experiments, "metrics_tcga.json")
    with open(path_aggc , 'r') as f:
        data_aggc = json.load(f)
    with open(path_tcga , 'r') as f:
        data_tcga= json.load(f)
    #这里需要收集

0.4919786096256685
0.6149732620320856
0.5935828877005348
0.5989304812834224
0.6203208556149733
0.6096256684491979
0.6096256684491979
0.6310160427807486
0.4385026737967914
0.40641711229946526
0.5882352941176471
0.37433155080213903
